## **Importer les librairies nécessaires**

In [1]:
import re
import requests

import pandas as pd
import numpy  as np

from datetime   import datetime
from bs4        import BeautifulSoup

## **Extraction**

#### 1. **Extraction des url pour chacun des articles présents sur la page**

In [2]:
# Requesting
url = 'https://www2.hm.com/en_us/men/products/jeans.html' # h&m url
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'} # user agent
page = requests.get(url, headers=headers) 

# BeautifulSoup Object
soup = BeautifulSoup(page.text, 'html.parser')

Du fait, que tous les articles ne s'affiche pas sur une seule et même page, nous devons trouver la taille de page maximale.

In [3]:
# Paging
total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total') # nombre d'items disponibles

page_number = np.round(int(total_item)/ 36) + 1 # nombre de page nécessaire (arrondi) 
url_complete = url + '?page-size=' + str(int(page_number*36)) # nouvelle url avec tous les items sur la même page

# Nouvelle requête
url = url_complete # url complete du site H&M
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser') 
products = soup.find('ul', class_='products-listing small') # trouver la liste complète des produits présents sur le site via la balise ul ayant pour classe 'products-listing small'

Cela nous permets de collecter les informations concernant l'identifiant du produits pour la suite, ainsi que du type de produit, puisque ces informations ne sont pas disponibles sur les liens de la fiche produits

In [4]:
# Récupération des informations
product_list = products.find_all('article', class_='hm-product-item') # on obtient chaque produit via la balise article et la classe 'hm-product-item'

# product id
product_id = [p.get('data-articlecode') for p in product_list]

# product_type
product_type = [p.get('data-category') for p in product_list]

# Construction du tableau initial
data = pd.DataFrame([product_id, product_type]).T
data.columns = ['product_id', 'product_type']

# Ajustement initiaux
data['product_id'] = data['product_id'].astype(str)
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data['color_id'] = data['product_id'].apply(lambda x: x[-3:])

#### 2. **Extraction d'autres attributs pour chaque produit présent sur le site internet**

In [ ]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'} # user agent

# Tableau de données de référence vide
df_compositions = pd.DataFrame()

# Requête API
for i in range(len(data)):
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    page = requests.get(url, headers=headers)

    soup = BeautifulSoup(page.text, 'html.parser') 

    ############################################################################################ Extraction des couleurs ########################################################################################
    id_color_list = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')

    color_name = [p.get('data-color') for p in id_color_list] # obtention des couleurs
    product_id = [p.get('data-articlecode') for p in id_color_list] # obtention des ID - Chaque couleur à un identifiant unique ID (3 derniers chiffres)

    df_color = pd.DataFrame([product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']

    for j in range(len(df_color)):
        ####################################### Requête API #######################################
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
        page = requests.get(url, headers=headers)    
        soup = BeautifulSoup(page.text, 'html.parser') 
        
        ####################################### Nom du produit #######################################
        product_name = soup.find_all('hm-product-name', id='js-product-name')
        product_name = product_name[0].get_text().strip()
        
        product_price = soup.find_all('div', class_='price parbase')
        product_price = product_price[0].get_text().strip()

        ######################################################################################## Extraction de la composition des articles #############################################################################
        product_composition_list = soup.find_all('div', class_='details-attributes-list-item') # attributes list
        product_composition = [list(filter(None, item.get_text().split('\n'))) for item in product_composition_list]
        
        if product_composition != []: # si l'information relation a la composition n'est pas vide
            df_composition_ref = pd.DataFrame(product_composition).T # On crée un tableau à partir de la liste product_composition
            df_composition_ref.columns = df_composition_ref.iloc[0, :] # Définir la première comme une colonne
            df_composition = df_composition_ref[['Fit', 'Composition', 'Art. No.']] # Sélectionner que les colonnes nécessaires
            df_composition = df_composition[df_composition['Composition'].notnull()]
            df_composition = df_composition.iloc[1:].fillna(method='ffill') # Gestion des valeurs nulles

            # 
            df_composition['Composition'] = df_composition['Composition'].replace('Shell: ', '', regex=True)

            # Renommer les  colonnes
            df_composition = df_composition.rename(columns = {'Fit' : 'fit', 'Composition' : 'composition', 'Art. No.' : 'product_id'})

            # Ajout des colonnes product_name et product_price
            df_composition['product_name'] = product_name
            df_composition['product_price'] = product_price
            ######################################################################################## Fusion #####################################################################################################
            # Color + Composition
            df_composition = pd.merge(df_composition, df_color, how='left', on='product_id')

            # Attributs
            df_compositions = pd.concat([df_compositions, df_composition], axis=0)
        
        else: # Si c'est vide
            None

# Génére Style ID + Color ID
df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3]) # product_id = style_id + color_id
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

# sDate et heure du scrapping
df_compositions['scraping_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')        

# Fusion
df_raw = pd.merge(data[['product_type', 'style_id']], df_compositions, how='left', on='style_id')

#### 3. **Fusion de product_type et df_compositions**

In [ ]:
df_raw = pd.merge(data[['product_type', 'style_id']], df_compositions, how='left', on='style_id')

## 3. **Nettoyage du jeu de données**

In [ ]:
df_raw

In [ ]:
print(df_raw.dtypes)
print()
print(df_raw.isna().sum())

In [ ]:
# Renommer les colonnes
df_raw.rename(columns = {'fit' : 'product_fit', 'color_name' : 'product_color', 'composition' : 'product_composition'}, inplace=True) 

In [ ]:
df_raw['product_color'] = df_raw['product_color'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

In [ ]:
df_raw['product_fit'] = df_raw['product_fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

In [ ]:
df_raw['product_name'] = df_raw['product_name'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

In [ ]:
df_raw['product_price'] = df_raw['product_price'].apply(lambda x: x.replace('$', ' ') if pd.notnull(x) else x)
df_raw['product_price'] = pd.to_numeric(df_raw['product_price'])

In [ ]:
df_raw['scraping_datetime'] = pd.to_datetime(df_raw['scraping_datetime'])

###**product_composition**

###### Séparation des éléments contenus dans la colonnes product_composition (i.e : Polyamide 64%, Spandex 36%)
###### d'autre part, on supprime certaines lignes (i.e: Pocket Lining")

In [ ]:
# Suppression de toutes les lignes contenant Pocket lining
df_raw = df_raw[~df_raw['product_composition'].str.contains('Pocket lining', na=False)] 

# Suppression de toutes les lignes contenant Lining
df_raw = df_raw[~df_raw['product_composition'].str.contains('Lining', na=False)] 

# Suppression de toutes les lignes contenant Pocket
df_raw = df_raw[~df_raw['product_composition'].str.contains('Pocket', na=False)].reset_index().drop(columns=['index']) 


df_aux = df_raw['product_composition'].str.split(',', expand=True) # Colonne auxiliaire

In [ ]:
df_aux[0].value_counts()

In [ ]:
df_aux[1].value_counts()

In [ ]:
df_aux[2].value_counts()

####Récupération de chaque composition et on les mets dans les bonnes colonnes

In [ ]:
# Cotton
# On met des valeurs de coton sur la colonne 'coton' si la première colonne de df1 contient des 'Coton', sinon on mets des NaN
df_aux['cotton'] = np.where(df_aux[0].str.contains('Cotton'), df_aux[0], np.nan) 
df_aux['cotton'] = np.where(df_aux[1].str.contains('Cotton'), df_aux[1], df_aux['cotton']) # il y a des valeurs de cotons sur la deuxième colonne

#  Spandex
# On met des valeurs de spandex sur la colonne 'spandex'si la deuxième colonne de df1 contient des 'Spandex', sinon on mets des NaN
df_aux['spandex'] = np.where(df_aux[1].str.contains('Spandex'), df_aux[1], np.nan) # il y a des valeurs de 'spandex'' sur la deuxième colonne
df_aux['spandex'] = np.where(df_aux[2].str.contains('Spandex'), df_aux[2], df_aux['spandex']) # il y a des valeurs de 'spandex'' sur la troisième colonne

# Polyester
df_aux['polyester'] = np.where(df_aux[1].str.contains('Polyester'), df_aux[1], np.nan) # il y a des valeurs de 'polyester'' sur la deuxième colonne

# Elastomultiester 
# On met des valeurs de polyester sur la colonne 'polyester'si la deuxième colonne de df1 contient des 'elastomultiester', sinon on mets des NaN
df_aux['elastomultiester'] = np.where(df_aux[1].str.contains('Elastomultiester'), df_aux[1], np.nan) 

# Lyocell
df_aux['lyocell'] = np.where(df_aux[0].str.contains('Lyocell'), df_aux[0], np.nan) 
df_aux['lyocell'] = np.where(df_aux[1].str.contains('Lyocell'), df_aux[1], df_aux['lyocell']) 

# Rayon
# On met des valeurs de rayon sur la colonne 'rayon'si la deuxième colonne de df1 contient des 'Rayon', sinon on mets des NaN
df_aux['rayon'] = np.where(df_aux[0].str.contains('Rayon'), df_aux[0], np.nan) 
df_aux['rayon'] = np.where(df_aux[2].str.contains('Rayon'), df_aux[2], df_aux['rayon']) # il y a des valeurs de 'rayon'' sur la troisième colonne

In [ ]:
# Nettoyage

# On supprime les colonnes temporaires
df_aux = df_aux.drop(columns=[0, 1, 2]) 

# On concatene df_aux et df_raw
df_raw = pd.concat([df_raw, df_aux], axis=1) 

# On supprime la colonne product_composition
df_raw = df_raw.drop(columns=['product_composition']) 

# Extraction uniquement des nombres présents sur chaque fiche produits
df_raw['cotton'] = df_raw['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else 0) 
df_raw['spandex'] = df_raw['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else 0)
df_raw['polyester'] = df_raw['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else 0) 
df_raw['elastomultiester'] = df_raw['elastomultiester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else 0) 
df_raw['lyocell'] = df_raw['lyocell'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else 0) 
df_raw['rayon'] = df_raw['rayon'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else 0) 



In [ ]:
df = df_raw.drop_duplicates().copy() # suppression des doublons

#On réarrange les colonnes pour que cela soit plus lisibles
df = df[['product_id', 'style_id', 'color_id', 'product_name', 'product_type', 
         'product_color',  'cotton', 'spandex', 'polyester',
         'elastomultiester', 'lyocell', 'rayon', 'product_price', 'scraping_datetime']] # rearranging columns


In [ ]:
# Le tableau final avec toutes les modifications apportées 
# On affiche que les 10 premiers élements du tableau
df.head(9)

In [ ]:
print(df.dtypes)
print()
print(df.isna().sum())

In [ ]:
from google.colab import files
df.to_csv('h&m_jeans_homme.csv', encoding='utf-8')
files.download('h&m_jeans_homme.csv')